In [68]:
import sys
import pickle
import os
import matplotlib.image as img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage.io import imread, imshow
from skimage import filters
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import pandas
import multiprocessing
from scipy.cluster.vq import vq,kmeans,whiten
import glob
import random
import shutil
from keras.preprocessing import image

from keras.applications.vgg16 import preprocess_input
from keras.models import load_model
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin 
from sklearn.neighbors import KNeighborsClassifier


In [69]:
module_root_name = "AgeEstimator"
module_paths = [
    os.path.abspath(os.path.join('..')),
    os.path.abspath(os.path.join('../..')),
    os.path.abspath(os.path.join('../../..'))
]
module_paths = list(filter(lambda x: x.endswith(module_root_name), module_paths))
module_path = module_paths[0] if len(module_paths) == 1 else ""
if module_path not in sys.path:
    sys.path.append(module_path)

In [70]:
from server.data.dataset import DataLoader

In [71]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=True)

def getPic(x_train):
    vgg16_feature_list = []
    for i in range(0, len(x_train)):
        pic = image.load_img(x_train[i], target_size=(224, 224))
        img_data = image.img_to_array(pic)
        
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        vgg16_feature = model.predict(img_data)
        vgg16_feature_np = np.array(vgg16_feature)
        vgg16_feature_list.append(vgg16_feature_np.flatten())
    vgg16_feature_list_np = np.array(vgg16_feature_list)
    #data=whiten(train)
    np.save("vgg16.npy",vgg16_feature_list_np)
    print("load image done")
    return vgg16_feature_list_np


In [72]:
def PCAnalysis(vgg16_feature_list_np):    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=0.9)
    pca_list = pca.fit_transform(vgg16_feature_list_np)
    print("PCA done")
    return pca_list

In [73]:
def GNB_train(trainPoints, trainLabels):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB().fit(trainPoints, trainLabels)
    with open('gnb.pickle', 'wb') as f:
        pickle.dump(gnb, f)
    print("GNB done")

In [74]:
def SVC_train(trainPoints, trainLabels):
    from sklearn.svm import SVC
    svclassifier = SVC(kernel='linear').fit(trainPoints, trainLabels)
    with open('svc.pickle', 'wb') as f:
        pickle.dump(svclassifier, f)
    print('SVC done')

In [75]:
def knn_train(trainPoints, trainLabels):
    k = 1000
    knn = KNeighborsClassifier(n_neighbors=int(k), metric='euclidean').fit(trainPoints, trainLabels)
    with open('knn.pickle', 'wb') as f:
        pickle.dump(knn, f)
    print("knn done")

In [76]:
def mbk_means_train(pca_list):
    
    mbk = MiniBatchKMeans(n_clusters=20, init='k-means++', max_iter=100, batch_size=100, verbose=0, compute_labels=True, random_state=None, 
                    tol=0.0, max_no_improvement=10, init_size=None, n_init=3, reassignment_ratio=0.01).fit(pca_list)
    
    mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis = 0) 
    mbk_means_labels = pairwise_distances_argmin(pca_list, mbk_means_cluster_centers) 
    print("mbk-means done")
    
    with open('mbk.pickle', 'wb') as f:
        pickle.dump(mbk, f)
    '''
    with open('mbk_test.pickle', 'wb') as f:
        pickle.dump(mbk, f)
    '''
    return mbk_means_labels

In [77]:
def k_mean_train(pca_list):
    kmeans = KMeans(n_clusters=20).fit(pca_list)
    centroids = kmeans.cluster_centers_
    k_means_labels = pairwise_distances_argmin(pca_list, centroids) 
    print("k-means done")
   
    with open('kmean.pickle', 'wb') as f:
        pickle.dump(kmeans, f)
    '''
    with open('kmean_test.pickle', 'wb') as f:
        pickle.dump(kmeans, f)
    '''
    return k_means_labels

In [78]:
def write(labels,model):
    
    fp = open(model+'_result.dat', 'w')
    i = 0
    for res in labels:
        fp.write(str(res))
        fp.write('\t')
        fp.write(str(y_train[i]))
        i = i + 1
        fp.write('\n')
    fp.close()

In [43]:

if __name__ == '__main__':
    dl = DataLoader()
    x_train, y_train = dl.load_train()
    x_test, y_test = dl.load_test()
    vgg16_feature_list_np = getPic(x_train)
    
    #vgg16_load = np.load("vgg16.npy")
    #print(len(vgg16_load))
    
    GNB_train(vgg16_feature_list_np,y_train)
    #SVC_tran(vgg16_feature_list_np,y_train)
    #knn_train(vgg16_load,y_train)
    #pca_list = PCAnalysis(vgg16_load)
    #mbk_means_labels = mbk_means_train(vgg16_feature_list_np)
    #k_mean_labels = k_mean_train(vgg16_feature_list_np)
    #write(mbk_means_labels,"mbk")
    #write(k_mean_labels,"kmean")

149724
GNB done
knn done


In [79]:
''' test
dl = DataLoader()
x_train, y_train = dl.load_train()
x_test, y_test = dl.load_test()
vgg16_load = np.load("vgg16.npy")
print(len(vgg16_load))
'''

149724


In [82]:
'''
loadImg = x_test[28]
#loadImg = 'C:\\Users\\USER\\Pictures\\picture.jpg'
pic = image.load_img(loadImg)
pic = pic.resize((224,224))
img_data = image.img_to_array(pic)
vgg16_feature_list = []
img_data = np.expand_dims(img_data, axis=0)
img_data = preprocess_input(img_data)

vgg16_feature = model.predict(img_data)
vgg16_feature_np = np.array(vgg16_feature)
test = vgg16_feature_np.flatten()
'''

In [83]:
'''
with open('svc.pickle', 'rb') as f:
    svc_load = pickle.load(f) 
    
with open('gnb.pickle', 'rb') as f2:
    gnb_load = pickle.load(f2) 

with open('knn.pickle', 'rb') as f3:
    knn_load = pickle.load(f3) 
print(loadImg)
print(svc_load.predict(vgg16_feature_np))
print(gnb_load.predict(vgg16_feature_np))
print(knn_load.predict(vgg16_feature_np))
'''

C:\Users\USER\Project\AgeEstimator\server\data\dataset\test\10_utk_21626.jpg
[26]
[8]
[26]
